In [1]:
import pandas as pd
import psycopg2 
import csv

df = pd.read_csv(r'/Users/caaturday/Documents/Projects/SQL/Corona Virus Project/CovidDeaths.csv')
df

,iso_code,continent,location,date,population,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests
0,AFG,Asia,Afghanistan,2/24/20,39835428.0,5.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2/25/20,39835428.0,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2/26/20,39835428.0,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2/27/20,39835428.0,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2/28/20,39835428.0,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185212,ZWE,Africa,Zimbabwe,5/6/22,15092171.0,248214.0,164.0,53.143,5473.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185213,ZWE,Africa,Zimbabwe,5/7/22,15092171.0,248214.0,0.0,48.429,5473.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185214,ZWE,Africa,Zimbabwe,5/8/22,15092171.0,248352.0,138.0,63.000,5476.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2272070.0
185215,ZWE,Africa,Zimbabwe,5/9/22,15092171.0,248536.0,184.0,85.857,5479.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
df.columns

Index(['iso_code', 'continent', 'location', 'date', 'population',
       'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths',
       'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests'],
      dtype='object')

In [3]:
df.dtypes

iso_code                               object
continent                              object
location                               object
date                                   object
population                            float64
total_cases                           float64
new_cases                             float64
new_cases_smoothed                    float64
total_deaths                          float64
new_deaths                            float64
new_deaths_smoothed                   float64
total_cases_per_million               float64
new_cases_per_million                 float64
new_cases_smoothed_per_million        float64
total_deaths_per_million              float64
new_deaths_per_million                float64
new_deaths_smoothed_per_million       float64
reproduction_rate                     float64
icu_patients                          float64
icu_patients_per_million              float64
hosp_patients                         float64
hosp_patients_per_million         

In [4]:
#processing data
replacements = {
    'timedelta64[ns]': 'varchar',
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp'
}

replacements 

{'timedelta64[ns]': 'varchar',
 'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp'}

In [5]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'iso_code varchar, continent varchar, location varchar, date varchar, population float, total_cases float, new_cases float, new_cases_smoothed float, total_deaths float, new_deaths float, new_deaths_smoothed float, total_cases_per_million float, new_cases_per_million float, new_cases_smoothed_per_million float, total_deaths_per_million float, new_deaths_per_million float, new_deaths_smoothed_per_million float, reproduction_rate float, icu_patients float, icu_patients_per_million float, hosp_patients float, hosp_patients_per_million float, weekly_icu_admissions float, weekly_icu_admissions_per_million float, weekly_hosp_admissions float, weekly_hosp_admissions_per_million float, total_tests float'

In [6]:
# connecting to the database
db_conn = psycopg2.connect("dbname=covid_project user=caaturday")
cur = db_conn.cursor()
print("Opened Database Successfully")

Opened Database Successfully


In [7]:
#insert values to table

#save df to csv
df.to_csv("CovidDeaths.csv", header=df.columns, index=False, encoding='utf-8')

#open the csv file, save it as an object
my_file = open("CovidDeaths.csv")
print('file opened in memory')

file opened in memory


In [8]:
# create table 
tbl_name = "CovidDeaths"


cur.execute("create table %s (%s);" % (tbl_name, col_str))
print('{0} was created successfully'.format(tbl_name)) 

CovidDeaths was created successfully


In [9]:
#upload to db
SQL_STATEMENT = """
COPY %s FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""
cur.copy_expert(sql=SQL_STATEMENT % tbl_name, file=my_file)
print('file copied to db')

file copied to db


In [13]:
db_conn.commit()
cur.close()
print('table {0} imported to db completed'.format(tbl_name))

table CovidDeaths imported to db completed
